In [1]:
import os
import pandas as pd
import nltk

In [2]:
dataDirectoryPath = 'resources/datasets-v5/tasks-2-3/train'
dataframesPath = 'resources'

In [3]:
from nltk import tokenize
from nltk.util import ngrams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import collections
from nltk.stem.porter import *
from nltk import word_tokenize
import string
import pickle
from resources.readability import Readability
from resources.feature_functions import Functions

In [4]:
def whatsbeendon(filename):
    pids = []
    try:
        with open(filename) as data:
            pids = [line.strip().split(",")[0] for line in data]
        return set(pids)
    except:
        return set(pids)

def make_str(seq):
    strseq = [str(s) for s in seq]
    return strseq

In [5]:
Functions = Functions()

In [6]:
outfile = "features_multi_classification.csv"
outpath = "resources/"

In [7]:
done = whatsbeendon(outfile)

In [8]:
cat_dict, stem_dict, counts_dict = Functions.load_LIWC_dictionaries()
liwc_cats = [cat_dict[cat] for cat in cat_dict]
pos_tags = ["CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNS","NNP","NNPS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","WP$","WRB","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP"]

In [9]:
if len(done) == 0:
    with open(os.path.join(outpath, outfile), "w") as out:
#         seq = ("pid, source, date, Happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, TTR,vad_neg,vad_neu,vad_pos,FKE,SMOG,stop,wordlen,WC,NB_pobj,NB_psubj,quotes,Exclaim,AllPunc,allcaps",",".join(pos_tags),",".join(liwc_cats))
        seq = ("ArticleId:start:stop, Happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, TTR,vad_neg,vad_neu,vad_pos,FKE,SMOG,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps",",".join(pos_tags),",".join(liwc_cats),"avg_word_sim,avg_sent_sim,x_y,x_2,y_2,PropagandaTechnique")
        out.write(",".join(seq)+"\n")

In [10]:
# dataset = pd.read_excel(dataframesPath + "/dataframemulti.xlsx")
dataset = pd.read_csv(dataframesPath + "/dataframemulti_with_simscores.csv")

In [11]:
dataset.head()

,Unnamed: 0,ArticleId,Start,End,Fragment,Technique,avg_word_sim,avg_sent_sim,x_y,x_2,y_2
0,0,111111112,191,221,Stop Islamization of America.,Slogans,0.065506,0.5215,0.041495,0.064896,0.330348
1,1,111111112,476,556,We condemn all those whose behaviours and view...,Black-and-White_Fallacy,0.339890,0.5215,0.215306,1.676765,0.330348
2,2,111111112,785,798,"Defeat Jihad""",Slogans,1.712280,0.5215,1.084657,14.397043,0.330348
3,3,111111112,958,1015,the nation that gave the world the Magna Carta...,Loaded_Language,0.719379,0.5215,0.455696,0.000000,0.330348
4,4,111111112,1456,1536,The UK should never become a stage for inflamm...,Flag-Waving,0.910841,0.5215,0.576979,0.000000,0.330348


In [12]:
dataset['ArticleId:start:stop'] = dataset['ArticleId'].astype(str) + ":" + dataset['Start'].astype(str) + ":" + dataset['End'].astype(str)

In [13]:
dataset = dataset[['ArticleId:start:stop', 'Fragment', 'avg_word_sim', 'avg_sent_sim', 'x_y', 'x_2', 'y_2', 'Technique']]

In [14]:
dataset.head()

,ArticleId:start:stop,Fragment,avg_word_sim,avg_sent_sim,x_y,x_2,y_2,Technique
0,111111112:191:221,Stop Islamization of America.,0.065506,0.5215,0.041495,0.064896,0.330348,Slogans
1,111111112:476:556,We condemn all those whose behaviours and view...,0.339890,0.5215,0.215306,1.676765,0.330348,Black-and-White_Fallacy
2,111111112:785:798,"Defeat Jihad""",1.712280,0.5215,1.084657,14.397043,0.330348,Slogans
3,111111112:958:1015,the nation that gave the world the Magna Carta...,0.719379,0.5215,0.455696,0.000000,0.330348,Loaded_Language
4,111111112:1456:1536,The UK should never become a stage for inflamm...,0.910841,0.5215,0.576979,0.000000,0.330348,Flag-Waving


In [15]:
len(dataset)

5107

In [16]:
x = dataset['Fragment']

In [17]:
x.head()

0                       Stop Islamization of America. 
1    We condemn all those whose behaviours and view...
2                                        Defeat Jihad"
3    the nation that gave the world the Magna Carta...
4    The UK should never become a stage for inflamm...
Name: Fragment, dtype: object

In [18]:
y = dataset['Technique']

In [19]:
y.head()

0                    Slogans
1    Black-and-White_Fallacy
2                    Slogans
3            Loaded_Language
4                Flag-Waving
Name: Technique, dtype: object

In [20]:
pos_features_path = "resources/temp/"

In [21]:
cat_dict, stem_dict, counts_dict = Functions.load_LIWC_dictionaries()
liwc_cats = [cat_dict[cat] for cat in cat_dict]

In [22]:
x[3454]

'totally conflicted'

In [23]:
pd.options.mode.chained_assignment = None

In [24]:
for pid in range(len(x)):
    if pid in done:
        continue
    else:
        print ((pid+1), " - working on", dataset['ArticleId:start:stop'][pid])
     
    quotes, Exclaim, AllPunc, allcaps = Functions.stuff_LIWC_leftout(dataset['ArticleId:start:stop'][pid], x[pid])
    lex_div = Functions.ttr(x[pid])
    counts_norm = Functions.POS_features("input", x[pid], pos_features_path)
    counts_norm = [str(c) for c in counts_norm]
    counts_norm_liwc, liwc_cats = Functions.LIWC(x[pid], cat_dict, stem_dict, counts_dict)
    counts_norm_liwc = [str(c) for c in counts_norm_liwc]
    vadneg, vadneu, vadpos = Functions.vadersent(x[pid])
    
    fke, SMOG = Functions.readability(x[pid])
    stop, wordlen, WC = Functions.wordlen_and_stop(x[pid])
#     print ("Functions.wordlen_and_stop(x[pid]) Done")
#     NB_pobj, NB_psubj = Functions.subjectivity(x[pid])
    bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count = Functions.bias_lexicon_feats(x[pid])
    HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral = Functions.moral_foundation_feats(x[pid])
    happiness = Functions.happiness_index_feats(x[pid])
    
    with open(os.path.join(outpath, outfile), "a", encoding="utf-8") as out:
        print ("writing out")
#         seq = (pid, cat, date, happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, lex_div,vadneg,vadneu,vadpos,fke,SMOG,stop,wordlen,WC,NB_pobj,NB_psubj,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc))
        seq = (dataset['ArticleId:start:stop'][pid], happiness, HarmVirtue, HarmVice, FairnessVirtue, FairnessVice, IngroupVirtue, IngroupVice, AuthorityVirtue, AuthorityVice, PurityVirtue, PurityVice, MoralityGeneral, bias_count, assertives_count, factives_count, hedges_count, implicatives_count, report_verbs_count, positive_op_count, negative_op_count, wneg_count, wpos_count, wneu_count, sneg_count, spos_count, sneu_count, lex_div,vadneg,vadneu,vadpos,fke,SMOG,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc),dataset['avg_word_sim'][pid],dataset['avg_sent_sim'][pid],dataset['x_y'][pid],dataset['x_2'][pid],dataset['y_2'][pid],y[pid])
#         seq = (pid, lex_div,vadneg,vadneu,vadpos,stop,wordlen,WC,quotes,Exclaim,AllPunc,allcaps, ",".join(counts_norm), ",".join(counts_norm_liwc))
        seq = make_str(seq)
        feat_str = ",".join(seq)
        out.write(feat_str + "\n")
    print("Processing complete")

1  - working on 111111112:191:221
writing out
Processing complete
2  - working on 111111112:476:556
writing out
Processing complete
3  - working on 111111112:785:798
writing out
Processing complete
4  - working on 111111112:958:1015
writing out
Processing complete
5  - working on 111111112:1456:1536
writing out
Processing complete
6  - working on 111111112:1810:1824
writing out
Processing complete
7  - working on 111111112:2095:2125
writing out
Processing complete
8  - working on 111111112:911:942
writing out
Processing complete
9  - working on 111111112:1738:1787
writing out
Processing complete
10  - working on 111111113:1396:1430
writing out
Processing complete
11  - working on 111111113:3082:3099
writing out
Processing complete
12  - working on 111111113:3828:3985
writing out
Processing complete
13  - working on 111111113:361:469
writing out
Processing complete
14  - working on 111111113:3580:3741
writing out
Processing complete
15  - working on 111111113:4865:5007
writing out
Proce

writing out
Processing complete
121  - working on 111111137:143:183
writing out
Processing complete
122  - working on 111111137:2058:2065
writing out
Processing complete
123  - working on 694327499:9221:9252
writing out
Processing complete
124  - working on 694327499:11573:11606
writing out
Processing complete
125  - working on 694327499:1366:1503
writing out
Processing complete
126  - working on 694327499:2798:2874
writing out
Processing complete
127  - working on 694327499:3032:3055
writing out
Processing complete
128  - working on 694327499:3656:3688
writing out
Processing complete
129  - working on 694327499:4811:4934
writing out
Processing complete
130  - working on 694327499:4944:4979
writing out
Processing complete
131  - working on 694327499:5950:6080
writing out
Processing complete
132  - working on 694327499:6320:6343
writing out
Processing complete
133  - working on 694327499:9393:9409
writing out
Processing complete
134  - working on 694327499:9630:9695
writing out
Processi

writing out
Processing complete
239  - working on 698018235:641:652
writing out
Processing complete
240  - working on 698018235:976:1049
writing out
Processing complete
241  - working on 698018235:1482:1493
writing out
Processing complete
242  - working on 698018235:3538:3578
writing out
Processing complete
243  - working on 698018235:3623:3651
writing out
Processing complete
244  - working on 698018235:976:1047
writing out
Processing complete
245  - working on 698018235:1910:1920
writing out
Processing complete
246  - working on 698018235:2132:2410
writing out
Processing complete
247  - working on 698018235:2431:2460
writing out
Processing complete
248  - working on 698018235:2723:2731
writing out
Processing complete
249  - working on 698018235:3283:3380
writing out
Processing complete
250  - working on 698018235:3514:3649
writing out
Processing complete
251  - working on 698092698:306:327
writing out
Processing complete
252  - working on 698092698:675:692
writing out
Processing compl

writing out
Processing complete
357  - working on 701837665:2828:2849
writing out
Processing complete
358  - working on 701837665:1364:1449
writing out
Processing complete
359  - working on 701837665:5040:5084
writing out
Processing complete
360  - working on 701837665:7193:7229
writing out
Processing complete
361  - working on 701837665:8276:8312
writing out
Processing complete
362  - working on 701837665:9299:9330
writing out
Processing complete
363  - working on 701837665:10112:10206
writing out
Processing complete
364  - working on 701837665:10208:10227
writing out
Processing complete
365  - working on 701837665:761:802
writing out
Processing complete
366  - working on 701837665:1829:2010
writing out
Processing complete
367  - working on 701837665:2534:2575
writing out
Processing complete
368  - working on 701837665:2618:2726
writing out
Processing complete
369  - working on 701837665:2792:2941
writing out
Processing complete
370  - working on 701837665:2828:2849
writing out
Proces

writing out
Processing complete
474  - working on 703821117:6445:6453
writing out
Processing complete
475  - working on 703821117:6520:6610
writing out
Processing complete
476  - working on 703821117:6620:6637
writing out
Processing complete
477  - working on 703821117:7264:7315
writing out
Processing complete
478  - working on 703821117:7674:7693
writing out
Processing complete
479  - working on 703821117:10469:10572
writing out
Processing complete
480  - working on 703821117:10572:10681
writing out
Processing complete
481  - working on 703821117:10697:10806
writing out
Processing complete
482  - working on 703821117:10740:10765
writing out
Processing complete
483  - working on 703821117:10860:10997
writing out
Processing complete
484  - working on 703821117:10820:10845
writing out
Processing complete
485  - working on 703821117:11221:11328
writing out
Processing complete
486  - working on 703821117:12149:12243
writing out
Processing complete
487  - working on 703821117:12149:12243
wr

...

Processing complete
4578  - working on 790266787:9833:9852
writing out
Processing complete
4579  - working on 790266787:10617:10651
writing out
Processing complete
4580  - working on 790266787:7693:7720
writing out
Processing complete
4581  - working on 790266787:8846:8882
writing out
Processing complete
4582  - working on 790266787:0:21
writing out
Processing complete
4583  - working on 790266787:99:133
writing out
Processing complete
4584  - working on 790266787:293:317
writing out
Processing complete
4585  - working on 790266787:346:365
writing out
Processing complete
4586  - working on 790266787:765:790
writing out
Processing complete
4587  - working on 790266787:1574:1585
writing out
Processing complete
4588  - working on 790266787:3035:3058
writing out
Processing complete
4589  - working on 790266787:3391:3531
writing out
Processing complete
4590  - working on 790266787:3561:3663
writing out
Processing complete
4591  - working on 790266787:3723:3737
writing out
Processing complet

Processing complete
4697  - working on 790677230:9784:9807
writing out
Processing complete
4698  - working on 790677230:10043:10068
writing out
Processing complete
4699  - working on 790677230:10304:10355
writing out
Processing complete
4700  - working on 790677230:866:886
writing out
Processing complete
4701  - working on 790677230:1121:1145
writing out
Processing complete
4702  - working on 790677230:1964:1973
writing out
Processing complete
4703  - working on 790677230:2144:2171
writing out
Processing complete
4704  - working on 790677230:4513:4620
writing out
Processing complete
4705  - working on 790677230:5626:5636
writing out
Processing complete
4706  - working on 790677230:7400:7497
writing out
Processing complete
4707  - working on 790677230:8857:8869
writing out
Processing complete
4708  - working on 790677230:9564:9568
writing out
Processing complete
4709  - working on 790677230:9666:9762
writing out
Processing complete
4710  - working on 790720480:422:435
writing out
Proces

writing out
Processing complete
4814  - working on 795703371:3386:3402
writing out
Processing complete
4815  - working on 795703371:3459:3469
writing out
Processing complete
4816  - working on 795703371:4103:4111
writing out
Processing complete
4817  - working on 795703371:3711:3748
writing out
Processing complete
4818  - working on 795703371:1268:1279
writing out
Processing complete
4819  - working on 999000145:247:384
writing out
Processing complete
4820  - working on 999000145:1502:1516
writing out
Processing complete
4821  - working on 999000145:2251:2319
writing out
Processing complete
4822  - working on 999000145:2393:2447
writing out
Processing complete
4823  - working on 999000145:2677:2703
writing out
Processing complete
4824  - working on 999000147:219:233
writing out
Processing complete
4825  - working on 999000147:3241:3324
writing out
Processing complete
4826  - working on 999000147:219:233
writing out
Processing complete
4827  - working on 999000147:2559:2678
writing out


writing out
Processing complete
4931  - working on 999001296:384:464
writing out
Processing complete
4932  - working on 999001296:466:560
writing out
Processing complete
4933  - working on 999001296:2509:2712
writing out
Processing complete
4934  - working on 999001296:2537:2584
writing out
Processing complete
4935  - working on 999001296:2596:2617
writing out
Processing complete
4936  - working on 999001296:3635:3652
writing out
Processing complete
4937  - working on 999001296:2946:3341
writing out
Processing complete
4938  - working on 999001296:165:173
writing out
Processing complete
4939  - working on 999001296:292:321
writing out
Processing complete
4940  - working on 999001296:475:482
writing out
Processing complete
4941  - working on 999001296:4687:4758
writing out
Processing complete
4942  - working on 999001297:2257:2277
writing out
Processing complete
4943  - working on 999001297:31:63
writing out
Processing complete
4944  - working on 999001297:242:264
writing out
Processing

writing out
Processing complete
5049  - working on 999001621:8377:8738
writing out
Processing complete
5050  - working on 999001621:8596:8627
writing out
Processing complete
5051  - working on 999001621:10904:10929
writing out
Processing complete
5052  - working on 999001621:5890:5903
writing out
Processing complete
5053  - working on 999001621:9006:9058
writing out
Processing complete
5054  - working on 999001621:0:68
writing out
Processing complete
5055  - working on 999001621:321:339
writing out
Processing complete
5056  - working on 999001621:467:501
writing out
Processing complete
5057  - working on 999001621:537:561
writing out
Processing complete
5058  - working on 999001621:279:599
writing out
Processing complete
5059  - working on 999001621:769:783
writing out
Processing complete
5060  - working on 999001621:1458:1495
writing out
Processing complete
5061  - working on 999001621:1332:1603
writing out
Processing complete
5062  - working on 999001621:2747:2771
writing out
Process